## LSI Search for related documents based on TF-IDF

[Benefit of LSI](https://en.wikipedia.org/wiki/Latent_semantic_analysis#Benefits_of_LSI): LSI overcomes two of the most problematic constraints of Boolean keyword queries: multiple words that have similar meanings (synonymy) and words that have more than one meaning (polysemy).

First, we define the similarity score by cosine similarity, then we construct the query vector based on the tf_idf matrix, later by reducing the dimension, we get a k dimension concept space. From that concept space, we can rank the relevance score between the query and each document, thus return the most relevant documents. 

<font color="blue"/>

### dsp:
  * &#x1f642; Brief but nice description.
  * "k dimension concept space" ~> "k dimensional concept space"
  * "we can rank the relevance score between the query and each document" ~> " we can calculate a relevance scores for the query and each document"
  * In the code below: There should be a space after a comma.
  * &#x1f632; Oops. I just followed your link "Benefit of LSI" and recognized that you copied the sentence verbally from Wikipedia without giving the reference and showing that it is a quote. If you need help on quoting. Here is what a quick Google search gave me: https://writing.wisc.edu/Handbook/QPA_quoting.html

#### 1. Run 04-TF-IDF_Raw_Implementation.ipynb first, to make sure tf-idf implementation can be invoked

In [1]:
%run 04-TF-IDF_Raw_Implementation.ipynb

........................................................................................................................................................................................................

CPU times: user 1.95 s, sys: 232 ms, total: 2.18 s
Wall time: 908 ms


In [ ]:
from scipy import spatial
import numpy as np
from numpy import linalg
import matplotlib.pyplot as plt
from matplotlib import colors

<font color="blue"/>

### dsp:
  * Better sort order: 1) numpy, 2) scipy, 3) matplotlib

In [3]:
#defining the similarity function
def similarity(u,v):
    return 1 - spatial.distance.cosine(u, v)

#### Find most related documents according to a list of keywords
- params: 
    - doc_sim: dict, key:index of the document, value:similarity of this specific document to those keywords
    - index2document: dict, key:document name (str), value:index (int)
- variables: 
    - related_doc: dict, key:document name, value: similarity of this document to those keywords

In [ ]:
def get_doc_relevance(doc_sim,index2document):
    related_doc = {}
    for doc,sim in sorted(doc_sim.items(),key = lambda x:x[1], reverse=True):
        related_doc[index2document[doc]] = sim
    return related_doc

<font color="blue"/>

### dsp:
  * I am surprised that a Python dictionary keeps the insertion order, but you seem to be right about that. &#x1f642;

#### 2. Run LSI
Latent semantic analysis  is a technique in natural language processing, in particular distributional semantics, of analyzing relationships between a set of documents and the terms they contain by producing a set of concepts related to the documents and terms. 

- params: 
    - q: query (str)
    - k: k-concept space
    - tf_idf: matrix obtained after invoking function: tf_idf_PrerocessedDoc(processed_doc_path, 200)
    - word2index: dict, obtained after invoking function: tf_idf_PrerocessedDoc(processed_doc_path, 200)
    - index2documtn: dict, obtained after invoking function: tf_idf_PrerocessedDoc(processed_doc_path, 200)
- variables: 
    - u, s, v: result of Singular value decomposition on tf-idf matrix
    - q_v: query vector -- q is mapped to the k-concept space,
        
        $q_v=q_v^{T}U_k\Sigma^{-1}_{k}$
        
    - doc_similarity: dict, key:index of the document, value:similarity of this specific document to those keywords

<font color="blue"/>

### dsp:
  * &#x1f642; Nice to have LSI as well.
  * You should describe the parameters independently of other functions, otherwise the user will not be able to use the function without using `tf_idf_PrerocessedDoc`. You may nevertheless mention that `tf_idf_PrerocessedDoc` is one source for the required parameters.
  * Is this $q_v$ on both sides of the equation?

In [ ]:
def latent_semantic_indexing(q, tf_idf, k, word2index, index2document):
    
    # preprocssing the query vector
    query = []
    doc_similarity = {}
    
    q_v = np.zeros(len(word2index))
    for w in q.split():
        try:
            q_v[word2index[w]]+=1
            query.append(word2index[w])
        except:
            print('keywords not found')
       
    u,s,v = linalg.svd(tf_idf)
    u = u[:,:k]
    s_ = np.zeros((k,k))
    
    for i in range(k):
        s_[i,i] = s[i]
    v = v[:k,:]
    q_v = np.zeros(len(word2index))
    
    for q in query:
        q_v[q]+=1
    q_v = q_v.reshape(1,-1)
    q_v = np.matmul(q_v,u)
    s_ = linalg.inv(s_)
    q_v = np.matmul(q_v,s_)
    
    for i in range(v.shape[1]):
        sim = similarity(q_v,v[:,i])
        doc_similarity[i] = sim 
    related_doc = get_doc_relevance(doc_similarity,index2document)
    
    return related_doc

<font color="blue"/>

### dsp:
  * You fill `q_v` twice.
  * You may write the matrix calcuation more similar to $q^{T}U_k\Sigma^{-1}_{k}$: With some care it should be `q.T.dot(u).dot(s.inv())`.
  * If a keyword is not found, a silent fail might be OK. But decide on your own.

In [6]:
%%time
#please modify the relative path
# processed_doc_path = '/home/bit/ma0/LabShare/data/chui_ma/spacy_corpus/'
processed_doc_path = '../spacy_corpus/'
tf_idf, word2index, index2document, inv_doc_freq = tf_idf_PrerocessedDoc(processed_doc_path, 200)

........................................................................................................................................................................................................

CPU times: user 2.65 s, sys: 268 ms, total: 2.92 s
Wall time: 1.42 s


In [7]:
%%time
#get related documents with relevance score
q = 'metzingen kollektion valentino permira '
related_doc = latent_semantic_indexing(q, tf_idf, 20, word2index, index2document)

CPU times: user 1min 55s, sys: 17.1 s, total: 2min 12s
Wall time: 24.9 s


#### Wrap the result with pandas DataFrame, and Coloring the table

In [ ]:
def getRelatedDocuments(related_doc):
    
    def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
        rng = M - m
        norm = colors.Normalize(m - (rng * low),
                                M + (rng * high))
        normed = norm(s.values)
        c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
        return ['background-color: %s' % color for color in c]
    
    df = pd.DataFrame(related_doc, index=['relevance score'])
    df = df.transpose()
    return df.style.apply(background_gradient,
                   cmap='PuBu',
                   m=df.min().min(),
                   M=df.max().max(),
                   low=0,
                   high=0.2)

<font color="blue"/>

### dsp:
  * I do not understand the name of the function. You could see this function as a generic method to display a dictionary containig numerical values.
  * The coloring did not work for me, but I did not run the Notebook.
  * &#x1f642; Great that you covered LSI as well.
  * Have you tried searching in paragraphs? It is hard to judge whether these documents are really related to the query. It might be more convincing to show paragraphs as search results.
  * Do a plain keyword search on your matches. If you actually find that "metzingen", "kollektion", "valentino", "permira" are _not_ in these documents, it would underline the importance of LSI.

In [9]:
getRelatedDocuments(related_doc)

,relevance score
WCM-QuarterlyReport-2015-Q2,0.991921
Linde-AnnualReport-2014,0.990486
alstria-QuarterlyReport-2016-Q2,0.99019
Wirecard-AnnualReport-2016,0.990038
Stroeer-AnnualReport-2010,0.989699
AmadeusFiRe-QuarterlyReport-2013-Q1,0.988369
alstria-QuarterlyReport-2011-Q1,0.987451
Jungheinrich-AnnualReport-2016,0.986979
alstria-AnnualReport-2016,0.986372
DialogSemiconductor-AnnualReport-2012,0.985887
